# Functional Programming in JavaScript - Experiences

<p class="biggg">Jakub Holý - Iterate conference 8/2015</p>

## Survey:

* JavaScript devs?
* Lo-dash / Underscore / etc.?
* Immutable.js etc.?
* React?

# 1. FP: What & Why?

```javascript
var localIp = _.chain(os.networkInterfaces())
   .values()
   .flatten()
   .filter(_.partial(supportsProtocol, "IPv4"))
   .map("address")
   .first()
   .value();
```

### NOTES

This is the embodiment of FP to me - similar to the picture of a person with skies and a cup of coffee waiting for a metro is the symbol of Oslo for me, even though both contain much more than this.

<p style="font-size: 6rem">```
Program: output = f(input)
```</p>

 * Side-effects minimized and limited (<=> immutable data)
 * Focus on data transformations
 * (WHAT vs. HOW)
 
=> **SIMPLICITY**

<p style="transform: rotate(180deg); text-align: right;font-size: xx-large">⇧</p>

1. Easier to understand and test
   * and cheaper b/c `costs = f exp(complexity)`
2. Referential transparency enables optimizations
   * memoize, parallelization, ...
3. Laziness, ∞ => only do what's needed, simpler code
4. Concurrency
5. Expressivity: complex transformations in few LoC

#### NOTES

* Referential transparency = fn can be replaced with its value without changing the behavior <> must be pure
* Pure fn - no side-effects, depends only on its inputs
* Lazy evaluation - eval sub-expressions only when you need the value; ex.: and(false, expensiveCall()) - expensiveCall will not be called

### What to expect from an FP lib

1. Functions: HOF, composing (comp), deriving (partial), running (iterate) fns
2. Data: Powerful fns for transformation of data structures

# 2. FP library: lodash-fp

In [19]:
var _ = require('lodash-fp');
var lodash = require('lodash');

undefined

## What you get

1. Functions: `chain, partial(Right), curry, spread, memoize, ...`
2. Data: `map, filter, reject, reduce, groupBy, mapValues, zip, ...`
3. Utilities for objects, sets, strings; `method, is*, ...`

## Lodash vs. lodash-fp

Data last, curryied =>

In [20]:
lodash.filter([1,2,3,4], function(num) { return _.gte(3, num); })

[ 3, 4 ]

In [6]:
_.filter( _.gte(3) )
        ( [1,2,3,4] )

[ 3, 4 ]

In [21]:
// Find obj with variations ∋ key "myID"
(lodash.flow(
     _.partialRight(_.result, "variations")  // -> {myID: true}
   , _.keys                                  // -> [myID]
   , _.partialRight(_.includes, "myID")      // -> true 
))
({variations: {myID: true}})

true

<p style="transform: rotate(180deg); text-align: right;font-size: xx-large">⇧</p>

In [12]:
(_.flow(
  _.result("variations"),
  _.keys,
  _.includes("myID")))
({variations: {myID: true}})

true

### SKIP: What we miss in lodash(-fp)

* Our own predicates other than `==`: `filter({age: _.partialRight(_.gt,18)}) // _.flow(_.result('age'),_.gt(18))`
* Immutable data to avoid have ugly nested `forEach` (`_.map` is cumbersome)
* Walk the tree and replace `{fields: obj, ..}` with `obj`
* `partition-with`, nested `map` (`employees.children`)
* Lookup (combine data from 2+ sources)
* Challenge: Split into rows, each of width 12: `[{width:12},{width:6},{width:6},.]`

## The dark side of lodash-fp

* No vararg fns, some split (uniq + uniqBy) => lost functionality
* => You need `lodash` & `lodash-fp` (size?!).

See [github/jakubholynet/lodash-fp_issues/.../lodash-fp_issues.ipynb](https://github.com/jakubholynet/lodash-fp_issues/blob/master/lodash-fp_issues.ipynb).

#### NOTES

In my experience you need ot keep lodash for the cases where the func isn't available in lodash-fp.
Not sure if including both increases the client-side size / how much.

Examples:

* `_.assign({}, {a:1}, {b:2})` is wrong - max 2
* `_.result` currently doesn't support a default value
* `zipObject([key, val, key2, val2, ..])` doesn't work, only `zipObject(keys, vals)`
* Iteratees such as `map(Values)`, `find` get only value, not the key argument
* Max does not accept the property argument: `_nofp.max([{n: 3},{n: 4}], \"n\") => { n: 4 }`

# 3. Immutable data

## Contestants

Immutable.js - Icepick - seamless-immutable

|                    | Size     | Maturity  |  API  | Plain JS?|
|--------------------|:--------:|:---------:|:-----:|:--------:|
| Immutable.js       |   56 kB  |    ★★★  | ★★★ |          |
| seamless-immutable |   3 kB   |     ★★   |  ★☆ |     ★    |
| icepick            |   3 kB   |     ★    |  ★★  |     ★   |

// `React.addon.update`

### Immutable.js

```javascript
Immutable([1,2,3,4,5])
  .skip(2)
  .map(n => -n)
  .filter(n => n % 2 === 0)
  .take(2).reduce((r, n) => r * n, 1);
```

* New data structures and API (`map.get("prop")`)
* ES3 / ES6
* lazy Seq, Set, Map, Ordered(Set|Map), Range, Record
* push, set, unshift,..; getIn, setIn, updateIn; `withMutations`
* Interop.: `toJS()`

## seamless-immutable

```javascript
Immutable([1,2,3]).concat([4]).map(double).filter(odd);
```

* Plain JS + `Object.freeze` (deep); ES5 (IE9+)
* Modifies existing methods (map, filter, ..) => worse interop
* Mutating methods throw an `ImmutableError` (`push`, `=`, etc.)
  <> map, filter, concat
* Minimalistic API: A: `flatMap, asObject, asMutable`; O: `merge, without, asMutable`
* Prod x dev build (freeze, exceptions)
* **∅ <del>updateIn, setIn</del>**


## icepick

```javascript
i.reverse(i.push(i.freeze([1,2,3], 4))
```

* Plain JS + `Object.freeze` (deep)
* A library of functions
* Richer API than s-i: assoc*, dissoc, get*, update*; push, splice, ..
* assign, merge,
* filter, map
* **∅ <del>reduce, map etc. over objects; chaining</del>**
* **one-man show**


### Immutable data conclusions

1. Use Immutable.js if you don't mind the size and don't need lot of interop (`to/fromJS`)
2. Use seamless-immutable + lodash-fp and write own convenience fn `updateIn` etc.
3. Or use icepick + lodash-fp if adventuresome or really need a particular [nested] element change (and write your `chain`)

## Immutable data & performance

Random benchmarks

* Immutable.js: Mutable ~900 ms | Immutable.js ×3 | I+shouldComponentUpdate ×0.7
* Seamless-immutable: Mutable: **TODO** | prod 800 ms | dev ×2-3

*Beware: The only benchmark that counts is yours, in your prod env.*

# 4. Conclusion 1: FP possible but painful; worth it?

1. Libraries quite OK but rough edges
   (other than `lodash-fp`?)
2. Immutable data: sub-par and/or large size & inconveniences

<table><tr><td>
You can make JavaScript fly...
<img src="flying-car.jpg" style="width: 500;height: 300px"/>
</td><td>
But why not to use something *designed* for flying?
<img src="plane-futuristic.jpg" style="width: 500;height: 300px"/>
</td></tr></table>


# Conclusion 2: Use ClojureScript

Why? Top-notch data functions and immutable data, core-async, macros, great design.

What about the bytes?



| JS: Nettbutikk | Cljs: Startshop|
|--------------------|--------|
| 300 kB      |   300 kB |
| React, router, lodash,.. |   Om, React, .. |
| No immutable, ... | Immutable, core-async, ..|
| (little own code) | (little own code) |

What about debugging? REPL! (And I live w/o debugger in JS anyway)


# Resources

* [The Power and Practicalities of Immutability](https://vimeo.com/131635253) - Venkat Subramaniam at NDC Oslo 2015
* [Immutable JavaScript](https://vimeo.com/128790457) - Christian Johansen at Web Rebels 2015 (Immutable.js & Minesweeper)
* [React: Rethinking best practices](https://www.youtube.com/watch?v=DgVS-zXgMTk) - Pete Hunt at JSConf 2013